# CHAPTER 04 다양한 분류 알고리즘

## 04-1 로지스틱 회귀



In [ ]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()  # head() : 처음 n개 행을 출력 (default: n=5)

In [ ]:
print(pd.unique(fish['Species']))  # unique('Feature') : 'Feature'열에서 고유한 값을 추출
# 타깃 값들

In [ ]:
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()  # 5개 열을 입력데이터로 사용
print(fish_input[:5])

fish_target = fish['Species'].to_numpy()  # 타깃 데이터 설정

In [ ]:
# 훈련세트와 테스트세트로 나누어줌
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [ ]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [ ]:
# k-최근접 이웃 분류기
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

**다중 분류** : 타깃 데이터에 2개 이상의 클래스가 포함된 문제

In [ ]:
print(kn.classes_)  # classes_ 속성: 정렬된 타깃값

print(kn.predict(test_scaled[:5]))  # 테스트세트의 첫 5개 샘플에 대한 예측

In [ ]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])  # 클래스별로 확률값을 반환
print(np.round(proba, decimals=4))  # 소수점 4번째 자리까지 표기

In [ ]:
distances, indexes = kn.kneighbors(test_scaled[3:4]) # 2차원 배열을 만들기 위해 슬라이싱 연산자 사용 (슬라이싱 사용 시 항상 2차원배열이 됨)
print(train_target[indexes])
# Roach 1개, Perch 2개 --> 2/3 확률 == 0.6667

###**로지스틱 회귀** :
 이름은 회귀이지만 분류 모델임

*z* = *a* x (Weight) + *b* x (Length) + *c* x (Diagonal) + *d* x (Height) + *e* x (Width) + *f*

- 시그모이드 함수 (로지스틱 회귀)
 
 *Φ* = 1 / ( 1 + e^(-z) )

 0 ~ 1 사이의 범위를 가지는 함수 --> 0 ~ 1 사이 값을 0~100% 확률로 해석

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

z = np.arange(-5,5,0.1)
phi = 1 / (1 + np.exp(-z))
plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

In [ ]:
char_arr = np.array(['A','B','C','D','E'])
print(char_arr[[True, False, True, False, False]])  # 1번째, 3번째 원소만 골라냄 (True인 index만)

In [ ]:
# 'Bream'과 'Smelt'행만 골라내는 작업
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]   # 'Bream'과 'Smelt' 만 있는 train_input
target_bream_smelt = train_target[bream_smelt_indexes]

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

print(lr.predict(train_bream_smelt[:5]))

In [ ]:
print(lr.predict_proba(train_bream_smelt[:5]))  # 첫 번째 열 : 음성클래스(0)
                                                # 두 번째 열 : 양성클래스(1)
print(lr.classes_)  # 알파벳순으로 정렬됨
                    # 'Bream' : 음성, 'Smelt' : 양성

In [ ]:
print(lr.coef_, lr.intercept_)  # 로지스틱 회귀가 학습한 계수

# z = -0.404 x (Weight) - 0.576 x (Length) - 0.663 x (Diagonal) - 1.013 x (Height) - 0.732 x (Weight) - 2.161

In [ ]:
decisions = lr.decision_function(train_bream_smelt[:5])  # '양성 클래스'에 대한 z값 반환
print(decisions)   # z 값들

In [ ]:
from scipy.special import expit  # expit() : 사이파이 라이브러리 안의 시그모이드 함수
print(expit(decisions))